In [ ]:
# Libraries to import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pickle
import geopandas
import folium
import json

In [ ]:
DATA_PATH = './data/'
DATA_PATH2020 = DATA_PATH + 'data_2020/'
FILE2020 = DATA_PATH2020 + 'M3_df_2020_final.pkl'


DATA_PATH = './data/'

DATA_PATH2015 = DATA_PATH + 'data2015/'
DATA_PATH2016 = DATA_PATH + 'data2016/'
DATA_PATH2017 = DATA_PATH + 'data2017/'
DATA_PATH2018 = DATA_PATH + 'data2018/'
DATA_PATH2019 = DATA_PATH + 'data2019/'
DATA_PATH2020 = DATA_PATH + 'data2020/'

FILE2015 = DATA_PATH2015 + 'M3_df_2015_final.pkl'
FILE2016 = DATA_PATH2016 + 'M3_df_2016_final.pkl'
FILE2017 = DATA_PATH2017 + 'M3_df_2017_final.pkl'
FILE2018 = DATA_PATH2018 + 'M3_df_2018_final.pkl'
FILE2019 = DATA_PATH2019 + 'M3_df_2019_final.pkl'
FILE2020 = DATA_PATH2020 + 'M3_df_2020_final.pkl'

FILES = [FILE2015, FILE2016, FILE2017, FILE2018, FILE2019, FILE2020]

In [ ]:
with open(FILE2020, 'rb') as input_file:
    df_2020_base = pickle.load(input_file)

In [ ]:
df_2020 = df_2020_base.copy()

In [ ]:
df_2020.columns

In [ ]:
with open(FILE2019, 'rb') as input_file:
    df_2019_base = pickle.load(input_file)

In [ ]:
df_2019 = df_2019_base.copy()

In [ ]:
def create_df_citizenship(df):
    df_citizenship = df.groupby(['gender', 'citizenship'])['numOcurrences'].sum().sort_values(ascending=False).to_frame(name='numOccurences').reset_index()
    return df_citizenship

## 2020

In [ ]:
df_2020

In [ ]:
df_cit_2020 = create_df_citizenship(df_2020)
df_cit_2020

In [ ]:
df_cit_2020 = df_cit_2020.set_index(['citizenship', 'gender']).sort_values(['citizenship','gender'])

In [ ]:
idx = pd.IndexSlice
df_cit_2020_f = df_cit_2020.loc[idx[:,'Female'],:].reset_index().drop("gender",axis=1)
df_cit_2020_f = df_cit_2020_f.rename({"numOccurences":"numOccurences_female"}, axis=1)
df_cit_2020_f

In [ ]:
idx = pd.IndexSlice
df_cit_2020_m = df_cit_2020.loc[idx[:,'Male'],:].reset_index().drop("gender",axis=1)
df_cit_2020_m = df_cit_2020_m.rename({"numOccurences":"numOccurences_male"}, axis=1)
df_cit_2020_m

In [ ]:
df_cit_2020_fm = pd.merge(df_cit_2020_m, df_cit_2020_f, on="citizenship", how='outer', sort=True)

In [ ]:
df_cit_2020_fm["female>male_2020"] = np.where(df_cit_2020_fm["numOccurences_male"]<df_cit_2020_fm["numOccurences_male"],1,0)

In [ ]:
df_cit_2020_fm

## 2019

In [ ]:
df_cit_2019 = create_df_citizenship(df_2019)

In [ ]:
df_cit_2019 = df_cit_2019.set_index(['citizenship', 'gender']).sort_values(['citizenship','gender'])

In [ ]:
idx = pd.IndexSlice
df_cit_2019_f = df_cit_2019.loc[idx[:,'Female'],:].reset_index().drop("gender",axis=1)
df_cit_2019_f = df_cit_2019_f.rename({"numOccurences":"numOccurences_female"}, axis=1)

In [ ]:
idx = pd.IndexSlice
df_cit_2019_m = df_cit_2019.loc[idx[:,'Male'],:].reset_index().drop("gender",axis=1)
df_cit_2019_m = df_cit_2019_m.rename({"numOccurences":"numOccurences_male"}, axis=1)

In [ ]:
df_cit_2019_fm = pd.merge(df_cit_2019_m, df_cit_2019_f, on="citizenship", how='outer', sort=True)

In [ ]:
df_cit_2019_fm["female>male_2019"] = np.where(df_cit_2019_fm["numOccurences_male"]<df_cit_2019_fm["numOccurences_male"],1,0)

In [ ]:
df_cit_2019_fm

## Combine all years

In [ ]:
df_cit_fm = pd.merge(df_cit_2019_fm[["citizenship", "female>male_2019"]],
                     df_cit_2020_fm[["citizenship", "female>male_2020"]], on = "citizenship", how='outer')

In [ ]:
df_cit_fm

## MAP

In [ ]:
with open('./data/countries.geojson', 'rb') as input_file:
    countries_json = json.load(input_file)

In [ ]:
my_map

In [ ]:
from branca.colormap import linear

my_map1 = folium.Map(tiles=None, overlay=False)


feature_group0 = folium.FeatureGroup(name='2019',overlay=False).add_to(my_map1)
feature_group1= folium.FeatureGroup(name='2020',overlay=False).add_to(my_map1)
fs = [feature_group0,feature_group1]
years = ["female>male_2019", "female>male_2020"]


for i in range(len(years)): 
    choropleth1 = folium.Choropleth(
    geo_data=countries_json,
    name='choropleth',
    data=df_cit_fm,
    columns=['citizenship', years[i]],
    key_on='feature.properties.ADMIN',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=1,
    line_opacity=1,
    #legend_name='',
    highlight=True,
    line_color='black').geojson.add_to(fs[i])
    
  

    #geojson for country labels
    geojson1 = folium.GeoJson(data=countries_json,
                              name='countries',
                              smooth_factor=2,
                              style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                              tooltip=folium.GeoJsonTooltip(fields=['ADMIN'],
                                                            labels=False,
                                                            sticky=True),
                              highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                             ).add_to(choropleth1)
    
folium.LayerControl(collapsed=False).add_to(my_map1)

'''
colormap = linear.YlGn_09.scale(0,1)
colormap.caption = 'Female speakers of citizenship more quoted than male'
colormap.add_to(my_map1)
'''


import branca.colormap as cmp
step = cmp.StepColormap(
 ['white', 'green'],
 vmin=0, vmax=1,
 index=[0,0.5,1],  #for change in the colors, not used fr linear
 caption='Female speakers of citizenship more quoted than male'
)
step.add_to(my_map1)



In [ ]:
my_map1